# Available Nested Samplers

Most nested sampling requires two things to generate samples from a posterior distribution:

1. A prior transform converting samples from a uniform distribution to the prior distribution
2. A log-likelihood function

RadVel's `Posterior` object implements both of these through `Posterior.prior_transform()` and `Posterior.likelihood_ns_array()`.
This means models defined with RadVel can be sampled with pretty much any nested sampling library.
For convenience, a few of them have wrappers implemented in the `radvel.nested_sampling` module:

- [UltraNest](https://johannesbuchner.github.io/UltraNest/index.html)
- [dynesty](https://dynesty.readthedocs.io/)
- [PyMultiNest](https://github.com/JohannesBuchner/PyMultiNest/)
- [Nautilus](https://nautilus-sampler.readthedocs.io/)

While these libraries all have a common API, they all have small differences in how they should be run and how they generate their outputs.
The `radvel.nested_sampling` module provides a common output format for all libraries, as explained below.

In this notebook, we will demonstrate how to sample a RadVel model with each of these libraries using the K2-24 example dataset from the [K2-24 Fitting & MCMC tutorial](https://radvel.readthedocs.io/en/latest/tutorials/K2-24_Fitting+MCMC.htm)

## Model Definition

First, we need to set up the RadVel model as we would do for MCMC.

In [ ]:
import time

import radvel
import numpy as np
from pandas import read_csv
import os
from radvel import nested_sampling as rns

import matplotlib.pyplot as plt

In [ ]:
# Set to True when to avoid re-running samplers
RESUME = True

In [ ]:
def initialize_model():
    time_base = 2420
    params = radvel.Parameters(2,basis='per tc secosw sesinw logk') # number of planets = 2
    params['per1'] = radvel.Parameter(value=20.885258)
    params['tc1'] = radvel.Parameter(value=2072.79438)
    params['secosw1'] = radvel.Parameter(value=0.01)
    params['sesinw1'] = radvel.Parameter(value=0.01)
    params['logk1'] = radvel.Parameter(value=1.1)
    params['per2'] = radvel.Parameter(value=42.363011)
    params['tc2'] = radvel.Parameter(value=2082.62516)
    params['secosw2'] = radvel.Parameter(value=0.01)
    params['sesinw2'] = radvel.Parameter(value=0.01)
    params['logk2'] = radvel.Parameter(value=1.1)
    mod = radvel.RVModel(params, time_base=time_base)
    mod.params['dvdt'] = radvel.Parameter(value=-0.02)
    mod.params['curv'] = radvel.Parameter(value=0.01)

    like = radvel.likelihood.RVLikelihood(mod, t, vel, errvel)
    like.params['gamma'] = radvel.Parameter(value=0.1, vary=False, linear=True)
    like.params['jit'] = radvel.Parameter(value=1.0)
    like.params['secosw1'].vary = False
    like.params['sesinw1'].vary = False
    like.params['per1'].vary = False
    like.params['tc1'].vary = False
    like.params['secosw2'].vary = False
    like.params['sesinw2'].vary = False
    like.params['per2'].vary = False
    like.params['tc2'].vary = False

    post = radvel.posterior.Posterior(like)
    post.priors += [radvel.prior.Gaussian('jit', np.log(3), 0.5)]
    post.priors += [radvel.prior.Gaussian('logk1', np.log(5), 10)]
    post.priors += [radvel.prior.Gaussian('dvdt', 0, 1.0)]
    post.priors += [radvel.prior.Gaussian('curv', 0, 1e-1)]
    post.priors += [radvel.prior.Gaussian('logk2', np.log(5), 10)]

    return post

In [ ]:
path = os.path.join(radvel.DATADIR,'epic203771098.csv')
rv = read_csv(path)

t = np.array(rv.t)
vel = np.array(rv.vel)
errvel = rv.errvel
ti = np.linspace(rv.t.iloc[0]-5,rv.t.iloc[-1]+5,100)

In [ ]:
post = initialize_model()
print(post)

In [ ]:
def plot_results(like):
    fig = plt.figure(figsize=(12,4))
    fig = plt.gcf()
    fig.set_tight_layout(True)
    plt.errorbar(
        like.x, like.model(t)+like.residuals(), 
        yerr=like.yerr, fmt='o'
        )
    plt.plot(ti, like.model(ti))
    plt.xlabel('Time')
    plt.ylabel('RV')
    return fig

In [ ]:
plot_results(post.likelihood)
plt.show()

## Nested Sampling

Now that we have our model, we can run one or more nested sampling algorithms.
Here we run all libraries available in RadVel.
Each library has its own `nested_sampling.run_<library_name>()` methods.

In their simplest form, all these methods require only a `Posterior` as input.
However, arguments can be passed directly to the samplers or the "run" functions through `sampler_kwargs` and `run_kwargs`, respectively.
We refer users to the documentation of each library (linked at the start of this notebook) to find out which arguments are available.

Each nested sampling function will return results in a standard dictionary format with the following keys:

- `samples`: _equally-weighted_ samples (equivalent to MCMC samples)
- `lnZ`: Natural log of the Bayesian evidence
- `lnZerr`: Statistical uncertainty on the evidence
- `sampler`: Sampler object from the underlying nested sampling package, providing acces to more details about the run and its extra outputs. This is note available for PyMultiNest, which stores the outputs on disk instead of in an Python object, by default.

## Running the samplers

Let us now run all available samplers and see how they perform in time.


In [ ]:
multinest_start = time.time()
multinest_results = rns.run(post, sampler="multinest", run_kwargs={"outputfiles_basename": "multinest_demo/out", "resume": RESUME})
multinest_time = time.time() - multinest_start
print(f"Running multinest took {multinest_time / 60:.2f} min")

In [ ]:
dynesty_start = time.time()
dynesty_results = rns.run(post, sampler="dynesty-static", run_kwargs={"checkpoint_file": "dynesty_demo/dynesty.save", "resume": RESUME})
dynesty_time = time.time() - dynesty_start
print(f"Running dynesty took {dynesty_time / 60:.2f} min")

In [ ]:
dynesty_start = time.time()
dynesty_dynamic_results = rns.run(post, sampler="dynesty-dynamic", run_kwargs={"checkpoint_file": "dynesty_demo_dynamic/dynesty_dynamic.save", "resume": RESUME})
dynesty_time = time.time() - dynesty_start
print(f"Running dynesty took {dynesty_time / 60:.2f} min")

In [ ]:
ultranest_start = time.time()
ultranest_results = rns.run(post, sampler="ultranest", sampler_kwargs={"log_dir": "ultranest_demo", "resume": RESUME})
ultranest_time = time.time() - ultranest_start
print(f"Running ultranest took {ultranest_time / 60:.2f} min")

In [ ]:
nautilus_start = time.time()
nautilus_results = rns.run(post, sampler="nautilus", sampler_kwargs={"filepath": "nautilus_demo/output.hdf5", "resume": RESUME}, run_kwargs={"verbose": True})
nautilus_time = time.time() - nautilus_start
print(f"Running nautilus took {nautilus_time / 60:.2f} min")

Once we have all the results, we can compare the Bayesian evidence and the posterior distributions to make sure they are consistent between samplers.
Keep in mind that nested sampling algorithms will perform differently on different problems.
Both speed and accuracy will depend on the number of dimensions (parameters) and the exact choice of settings.
See [Buchner (2021)](https://arxiv.org/abs/2101.09675) for a review of this topi..

In [ ]:
print(f"Multinest: {multinest_results['lnZ']:.2f} +/- {multinest_results['lnZerr']:.2f}")
print(f"Dynesty (static): {dynesty_results['lnZ']:.2f} +/- {dynesty_results['lnZerr']:.2f}")
print(f"Dynesty (dynamic): {dynesty_dynamic_results['lnZ']:.2f} +/- {dynesty_dynamic_results['lnZerr']:.2f}")
print(f"Ultranest: {ultranest_results['lnZ']:.2f} +/- {ultranest_results['lnZerr']:.2f}")
print(f"Nautilus: {nautilus_results['lnZ']:.2f} +/- {nautilus_results['lnZerr']**-0.5:.2f}")

In [ ]:
import corner

hist_kwargs = {"density": True}
fig = corner.corner(nautilus_results["samples"], labels=post.name_vary_params(),range=np.repeat(0.999, len(post.name_vary_params())), hist_kwargs=hist_kwargs)

corner.corner(multinest_results["samples"], color="b", fig=fig, hist_kwargs=hist_kwargs | {"color": "b"})

corner.corner(dynesty_results["samples"], color="r", fig=fig, hist_kwargs=hist_kwargs | {"color": "r"},range=np.repeat(0.999, len(post.name_vary_params())))

corner.corner(dynesty_dynamic_results["samples"], color="yellow", fig=fig, hist_kwargs=hist_kwargs | {"color": "yellow"},range=np.repeat(0.999, len(post.name_vary_params())))

corner.corner(ultranest_results["samples"], color="peachpuff", fig=fig, hist_kwargs=hist_kwargs | {"color": "peachpuff"}, range=np.repeat(0.999, len(post.name_vary_params())))

plt.show()